<a href="https://colab.research.google.com/github/ykato27/BERT-Japanese/blob/main/2_BERT_livedoor_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-07-03 05:32:47--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.2’

ldcc-20140209.tar.g 100%[===================>]   8.44M  5.12MB/s    in 1.6s    

2021-07-03 05:32:50 (5.12 MB/s) - ‘ldcc-20140209.tar.gz.2’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['CHANGES.txt', 'livedoor-homme', 'it-life-hack', 'topic-news', 'movie-enter', 'smax', 'sports-watch', 'dokujo-tsushin', 'peachy', 'README.txt', 'kaden-channel']
カテゴリー数: 9
['livedoor-homme', 'it-life-hack', 'topic-news', 'movie-enter', 'smax', 'sports-watch', 'dokujo-tsushin', 'peachy', 'kaden-channel']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [6]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [7]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


スポーツメーカー大手デサント社による人気ゴルフブランド「ルコックゴルフ」では、ゴルファーをサポートする数々のウェアやアイテムをリリースしているのはご存知だろう。中でも、今年ゴルファーの間で一際大きな話題を集めているのが「ウルビルド4D」なる裁断技術を使用した最新ゴルフウェアになる。まず、ウルビルドとはドイツ語で原型という意味を持つ。3Dならぬ4Dは「4ディメンション」（4次元）のことを指し、3Dの立体裁断は静体位で静止した身体をベースに裁断してウェアを制作しているのに対し、4Dでは動きを想定した裁断法を取り入れているというわけだ。これまでなら、スイングなどの動作は、ウェアによって身体の動きが制限されてしまうこともあったが、この「ウルビルド4D」シリーズでは、これを極限まで抑え、可動域を大幅に広げているのが特徴で、2Dのものに比べれば、脇部分が上下運動で27％、肩・背中部分が水平方向運動で73％、内股部分は56％、臀部〜ひざ部分屈伸運動では46％も拡大したという。また、ボトムスもウルビルド4Dを採用しており、普通のパンツに比べると、その履き心地はもちろん、4Dで裁断したパーツを使用することで、しゃがんだりひざを曲げたときにウエストの部分が浮いたりするのをおさえ、身体にフィットした抜群の動きやすさを実現している。さらに、トップス＆ボトムスに共通して、通気性が非常に良いことも見逃せない。これらはソニックエアという、動くと風通りがよくなる素材を使用しており、汗を吸ってすばやく乾かすのと同時に衣服内の熱気を外に出すという気化熱を応用しているため、暑い日差しが照りつける夏場のプレーも快適＆清潔に——。それでいて、プレーの質が向上する「ウルビルド4D」シリーズ、この夏のプレーでアナタも試してみてはいかがだろうか。■関連リンク・ルコックゴルフ公式サイト・ゴルフ特集 2010 - livedoor HOMME■関連記事・ビームスがオススメするゴルフウェアの現在 【ゴルフ特集】・ゴルフ女子が選ぶ、人気キャディバッグ「OGIO」トップ3 【ゴルフ特集】・ボールにも色気を！ スーパーブルーがあなたを救う 【ゴルフ特集】・カモフラ柄でミスショットも“カモフラージュ” 【ゴルフ特集】
livedoor-homme


In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,スポーツメーカー大手デサント社による人気ゴルフブランド「ルコックゴルフ」では、ゴルファーをサ...,livedoor-homme
1,ワークスペースのデザイニングに優れた企業のオフィスを訪問し、そこで働く人々の声から、その”働...,livedoor-homme
2,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...,livedoor-homme
3,世界的DJであるFATBOY SLIM公認のビーチパーティー「BIG BEACH FESTI...,livedoor-homme
4,フランスが世界に誇るブランド、エス・テー・デュポン社は、3月11日に発生した東日本大震災を受...,livedoor-homme


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'livedoor-homme', 1: 'it-life-hack', 2: 'topic-news', 3: 'movie-enter', 4: 'smax', 5: 'sports-watch', 6: 'dokujo-tsushin', 7: 'peachy', 8: 'kaden-channel'}
{'livedoor-homme': 0, 'it-life-hack': 1, 'topic-news': 2, 'movie-enter': 3, 'smax': 4, 'sports-watch': 5, 'dokujo-tsushin': 6, 'peachy': 7, 'kaden-channel': 8}


,text,label_index
0,スポーツメーカー大手デサント社による人気ゴルフブランド「ルコックゴルフ」では、ゴルファーをサ...,0
1,ワークスペースのデザイニングに優れた企業のオフィスを訪問し、そこで働く人々の声から、その”働...,0
2,「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽き...,0
3,世界的DJであるFATBOY SLIM公認のビーチパーティー「BIG BEACH FESTI...,0
4,フランスが世界に誇るブランド、エス・テー・デュポン社は、3月11日に発生した東日本大震災を受...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,日本テレビ「ダウンタウンDX」（16日放送分）では、福岡ソフトバンクホークスリハビリ担当コー...,5
1,ここ1〜2年、店内の一角にシールコーナを設けている文具店や雑貨店が増えている。一歩踏み込むと...,6
2,1985年に公開された映画『バック・トゥ・ザ・フューチャー』でタイムマシンとして登場し、デロ...,3
3,先日、携帯電話を買い替えるために家電量販店へ行ったところ、担当の販売員の方が、柳原可奈子にそ...,6
4,阪神タイガースの金本知憲外野手(43)の背番号6が、永久欠番に内定していると26日付のデイリ...,5


In [12]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [13]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [14]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.13)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

In [15]:
!sudo cp /etc/mecabrc /usr/local/etc/

In [16]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')
# # 日本語BERTの分かち書き用tokenizerです
# tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [17]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [18]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.legacy.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [19]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [20]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


tensor([    2,   599,     5,   640,     9,     6,  1568,  2032, 15872,    18,
         1628, 28452, 28512,     8, 12614,    11,  1330,    16,  2786,    84,
         3318,     6,  7052,     7, 13770,     6,   599,   271,    64,     6,
         8583,    15,    10,   640,    11,  4166,  2156,  4627,     8,   687,
        28536,    36,   599,  9551,     5,     9, 14596,    12,     6,  9749,
        21910, 28447, 14406, 28504,    58,  3051, 28489,    38,   140,    53,
            9,   707,     5,    12,     9, 17230,    55, 23749,    13,    15,
           10, 15221,    12, 19235,  3877,    18,   599,    11,  7138,    82,
            7,     9,     6,  1040,     5, 13253,    11,  7881,    34,    45,
           14, 11047,     8,  2909,  7168,     9,     6,  6724,  5464,    12,
        22465,     7,  7852,   392,     6, 11255, 11214,    35,  3265, 30161,
         4437,    11,  1951,    15,  2610,     8, 16062, 11255, 11214,  4437,
           12,  2333, 29630,   742, 28700,   679, 28496,  2340, 

In [21]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


['[CLS]', '朝', 'の', '時間', 'は', '、', '女子', 'にとって', '大事', 'な', 'ひと', '##と', '##き', '。', '余裕', 'を', '持っ', 'て', '起き', 'られ', 'たら', '、', '食事', 'に', 'メイク', '、', '朝', '活', 'など', '、', '充実', 'し', 'た', '時間', 'を', '過ご', '##せる', 'はず', '。', 'だけ', '##ど', '「', '朝', '起きる', 'の', 'は', '苦手', 'で', '、', 'いつも', '慌', '##た', '##だし', '##く', 'なっ', 'ちゃ', '##う', '」', 'という', '人', 'は', '多い', 'の', 'で', 'は', '!?', 'ス', '##ッキリ', 'と', 'し', 'た', '目覚め', 'で', 'さわ', '##やか', 'な', '朝', 'を', '迎える', 'ため', 'に', 'は', '、', '自分', 'の', '睡眠', 'を', 'コントロール', 'する', 'こと', 'が', '大切', '。', 'そこで', '今回', 'は', '、', 'スマート', 'フォン', 'で', '手軽', 'に', 'ダウンロード', 'できる', '、', '目覚', '##まし', '・', '快', '##眠', 'アプリ', 'を', '紹介', 'し', 'ます', '。', 'ユニーク', '目覚', '##まし', 'アプリ', 'で', '遅', '##刻', '知ら', '##ず', '!', '##「', 'もう', '普通', 'の', '目覚', '##まし', 'で', 'は', '起き', 'られ', 'ない', '!」', 'という', '人', 'に', 'オス', '##ス', '##メ', 'な', 'の', 'が', '、', 'スマート', 'フォン', 'の', '機能', 'を', '活かし', 'た', 'ユニーク', '目覚', '##まし', 'アプリ', '。', 'Twitter', 'や', 'f', '##ace', '##bo', '##ok', 

'peachy'

In [22]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.legacy.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


In [23]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)


[torchtext.legacy.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [24]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [25]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output


In [26]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [27]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [28]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [29]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [30]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.1614 || 10iter. || 本イテレーションの正解率：0.125
イテレーション 20 || Loss: 1.9659 || 10iter. || 本イテレーションの正解率：0.3125
イテレーション 30 || Loss: 1.8514 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 40 || Loss: 1.1241 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 50 || Loss: 1.2962 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 60 || Loss: 0.8704 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 70 || Loss: 0.9895 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 80 || Loss: 0.7007 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 90 || Loss: 0.5129 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 100 || Loss: 0.8516 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 110 || Loss: 0.7194 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 120 || Loss: 0.6843 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 130 || Loss: 0.9372 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 140 || Loss: 0.4547 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 150 || Loss: 0.3959 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 160 || Loss: 0.2990 || 10iter. || 本イテレーションの正解率：0.87

## テストデータでの性能を確認

In [31]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


100%|██████████| 93/93 [00:55<00:00,  1.68it/s]

テストデータ1475個での正解率：0.9024
